In [1]:
# Importamos librerías necesarias
%matplotlib inline
from brian2 import *
from brian2tools import *
import ipywidgets as ipw

# Usamos numpy como motor de cálculo
prefs.codegen.target = 'numpy'

# Definimos el modelo
def modelo(N=20, R_max=150, f=10, w=0.1, p=0.5, tau=5, tau_t=50, delta_t=0.1):
    # Parámetros de simulación
    R_max = R_max*Hz       # Frecuencia máxima de disparo
    f = f*Hz               # Frecuencia de modulación
    tau = tau*ms           # Constante de tiempo de membrana
    tau_t = tau_t*ms       # Constante de tiempo de adaptación
    duracion = 200*ms      # Tiempo total de simulación
    
    # Neuronas de entrada (Poisson)
    G = PoissonGroup(N, rates='R_max*0.5*(1+sin(2*pi*f*t))')
    
    # Neurona objetivo con adaptación
    ecuaciones = '''
    dV/dt = -V/tau : 1
    dVt/dt = (1-Vt)/tau_t : 1
    '''
    H = NeuronGroup(1, ecuaciones,
                    threshold='V>Vt',
                    reset='V=0; Vt += delta_t',
                    method='linear')
    H.Vt = 1  # valor inicial del umbral
    
    # Sinapsis entre neuronas de entrada y la neurona objetivo
    S = Synapses(G, H, on_pre='V += w')
    S.connect(p=p)  # conexión con probabilidad p
    
    # Monitores de actividad
    MG = SpikeMonitor(G)                  # Registra disparos de entrada
    MH = StateMonitor(H, ('V', 'Vt'), record=True)  # Registra voltaje y umbral
    
    # Ejecutamos la simulación
    run(duracion)
    
    # Graficamos resultados
    figure(figsize=(15, 4))
    
    # 1. Disparos de las neuronas fuente
    subplot(131)
    brian_plot(MG)
    title('Neuronas de entrada (Poisson)')
    
    # 2. Esquema de sinapsis
    subplot(132)
    plot(zeros(N), arange(N), 'ob')
    plot([0, 1], [S.i, ones(len(S.i))*N/2.], '-k')
    plot([1], [N/2.], 'og')
    xlim(-0.1, 1.1)
    ylim(-1, N)
    axis('off')
    title('Conexiones sinápticas')
    
    # 3. Voltaje de la neurona objetivo
    subplot(133)
    plot(MH.t, MH.V[0], label='Voltaje (V)')
    plot(MH.t, MH.Vt[0], label='Umbral (Vt)')
    legend(loc='upper left')
    title('Neurona objetivo')

# Widgets interactivos para variar parámetros en vivo
layout = ipw.Layout(width='100%')
style = {'description_width': 'initial'}

ipw.interact(modelo,
             N=ipw.IntSlider(value=20, min=5, max=100, step=5, continuous_update=False,
                             description="Número de neuronas de entrada", style=style, layout=layout),
             R_max=ipw.FloatSlider(value=300, min=0, max=500, step=10, continuous_update=False,
                                   description="Frecuencia máx. de entrada (Hz)", style=style, layout=layout),
             f=ipw.FloatSlider(value=10, min=1, max=50, step=1, continuous_update=False,
                               description="Frecuencia de modulación (Hz)", style=style, layout=layout),
             p=ipw.FloatSlider(value=0.5, min=0, max=1, step=0.01, continuous_update=False,
                               description="Probabilidad de conexión sináptica", style=style, layout=layout),
             w=ipw.FloatSlider(value=0.3, min=0, max=1, step=0.01, continuous_update=False,
                               description="Peso sináptico", style=style, layout=layout),
             tau=ipw.FloatSlider(value=5, min=1, max=50, step=1, continuous_update=False,
                                 description="Constante de tiempo de membrana (ms)", style=style, layout=layout),
             tau_t=ipw.FloatSlider(value=30, min=5, max=500, step=5, continuous_update=False,
                                   description="Constante de adaptación (ms)", style=style, layout=layout),
             delta_t=ipw.FloatSlider(value=1.0, min=0, max=20, step=0.1, continuous_update=False,
                                     description="Fuerza de adaptación", style=style, layout=layout),
            );


interactive(children=(IntSlider(value=20, continuous_update=False, description='Número de neuronas de entrada'…